In [1]:
import os
import time
import warnings
import pandas as pd
import numpy as np

import bokeh
from bokeh.io import output_notebook, show
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

import utils

warnings.filterwarnings('ignore')
pd.options.plotting.backend = "pandas_bokeh"

output_notebook()
bokeh.__version__

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

# Load data

In [2]:
# Default toy model
paths = {
    '01' : 'data/stream_01.csv'
}

streams = {s : pd.read_csv(p, dtype=int) for s, p in paths.items()}

In [3]:
# # Prophesee
# paths = {
#     '01' : '../data_prophesee/moorea_2019-02-18_000_td_2928500000_2988500000_td.csv'
# }

# streams = {s : pd.read_csv(p, dtype=int) for s, p in paths.items()}

# for s, df in streams.items():
#     df['y'] = 719 - df['y']

In [4]:
# # Stereo iniviation
# paths = {
#     '01' : '../data_inivation/stream_1.csv',
#     '02' : '../data_inivation/stream_2.csv'
# }
# streams = {s : pd.read_csv(p, dtype=int) for s, p in paths.items()}

In [5]:
streams['01'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   timestamp  100000 non-null  int64
 1   x          100000 non-null  int64
 2   y          100000 non-null  int64
 3   p          100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [6]:
streams['01'][:10]

,timestamp,x,y,p
0,0,100,150,0
1,1,98,150,0
2,2,105,154,1
3,3,99,141,1
4,4,104,154,1
5,5,100,148,1
6,6,99,149,0
7,7,106,151,1
8,8,94,146,0
9,9,104,153,1


In [7]:
timestamp_min = 1e99

for s, df in streams.items():

    print('\nStream: ', s)
    print('x_min:', df['x'].min())
    print('x_max:', df['x'].max())
    print('y_min:', df['y'].min())
    print('y_max:', df['y'].max())
    print('Timestamp_min:', df['timestamp'].min())
    print('Timestamp_max:', df['timestamp'].max())
    print('Timestamp diff:', df['timestamp'].max() - df['timestamp'].min())
    print('Average number of events per timestamp:',len(df.index)/(df['timestamp'].max() - df['timestamp'].min()))

    timestamp_min = min(df['timestamp'].min(), timestamp_min)


Stream:  01
x_min: 40
x_max: 160
y_min: 40
y_max: 160
Timestamp_min: 0
Timestamp_max: 99999
Timestamp diff: 99999
Average number of events per timestamp: 1.000010000100001


#### Set starting timestamp to zero. The code below assumes that the starting timestamp is 0.

In [8]:
for s, df in streams.items():

    df['timestamp'] -= timestamp_min
    
    print(df['timestamp'].min())

0


#### Load event annotations if they already exist. Put them in the same folder as original csv file and add "_anno.csv"

In [9]:
for s, df in streams.items():
    
    path_anno = paths[s].replace('.csv', '_anno.csv')

    if os.path.exists(path_anno):
        print('Loading: {}'.format(path_anno))
        df['anno'] = pd.read_csv(path_anno, dtype=int)

# Initial studies

#### Take sample of each stream 

In [10]:
num_sample = 100000
samples = {s:df.sample(n=num_sample) for s, df in streams.items()}

#### Lets see how is event polarity distributed as a function of a timestamp.

In [11]:
df_hist = pd.DataFrame({
    'Stream #1 (p=0)' : samples['01']['timestamp'][samples['01']['p']==0],
    'Stream #1 (p=1)' : samples['01']['timestamp'][samples['01']['p']==1],
});

df_hist.plot_bokeh(kind='hist', bins=100, title='Sample of events as function of timestamp.', xlabel='Timestamp', xticks=[]);

# Event video

In [14]:
event_tool = utils.EventTool(streams)

# bokeh part
app = Application(FunctionHandler(event_tool.app_function))
show(app)